In [ ]:
!pip install latex2sympy

In [ ]:
from itertools import combinations
import numpy as np
from latex2sympy import strToSympy
from collections import defaultdict
import graphviz
from sympy import Symbol, symbols, Eq, sympify
import networkx as nx
from more_itertools import powerset
import random
from itertools import combinations, chain
import matplotlib.pyplot as plt
import tqdm as tqdm

In [ ]:
class Equation:
    def __init__(self, formula) -> None:
        self.formula = formula
        self.equation = self.get_equation()
        self.vars = self.get_vars()
    
    def get_equation(self):
        equation = strToSympy(self.formula)
        return equation

    def get_vars(self):
        vars = sorted(self.equation.free_symbols, key=lambda symbol: symbol.name)
        return vars

In [ ]:
class Structure:
    def __init__(self, equations, vars=None):
        self.equations = equations
        if vars is not None:
            all_vars = set().union(*(map(lambda x: set(x.vars), self.equations)))
            subs_vars = all_vars.difference(vars)
            subs_vars_values = [(var, 0) for var in subs_vars]

            self.vars = vars
            equalities = list(map(lambda x: x.equation.subs(subs_vars_values), self.equations))
            for i in range(len(self.equations)):
                self.equations[i].equation = equalities[i]
        else:
            self.vars = set().union(*(map(lambda x: set(x.vars), self.equations)))

    @classmethod
    def generate_random(cls, num_vars=3, num_equations=2, count=False, max_terms=10, coef_range=(-5, 5), force_all_vars=True, max_attempts=100):
        '''
        Генерируем случайную структуру

        num_vars - количество переменных
        num_equations - количество выражений
        max_terms - максимальное количество членов в одной части уравнения
        coef_range - разброс коэффициентов перед переменными
        force_all_vars - флаг, использовать ли все переменные
        max_attempts - количество попыток генерации структуры
        '''
        if num_equations > num_vars:
            raise ValueError("Количество уравнений не может превышать количество переменных")
    
        if max_terms < 1:
            raise ValueError("max_terms должен быть не менее 1")
    
        if num_vars < 1:
            raise ValueError("Должна быть хотя бы одна переменная")
        
        # определяем доступные операции
        operations = ['+', '-', '*', '/']
        
        # создаем символы для переменных
        var_names = [f'x{i}' for i in range(num_vars)]
        vars = symbols(' '.join(var_names))
        if num_vars == 1:
            vars = [vars]
        
        attempt = 0
        while attempt < max_attempts:
            attempt += 1
            equations = []
            used_vars = set()
            
            if force_all_vars and num_equations > 0:
                eq_vars = random.sample(list(vars), min(num_vars, max(1, max_terms)))
                lhs = cls._generate_expression(eq_vars[:len(eq_vars)//2], operations, coef_range)
                rhs = cls._generate_expression(eq_vars[len(eq_vars)//2:], operations, coef_range)
                equations.append(Equation(f"{lhs} = {rhs}"))
                used_vars.update(eq_vars)
            
            for _ in range(len(equations), num_equations):
                if force_all_vars and (remaining_vars := set(vars) - used_vars):
                    num_new = random.randint(1, min(max_terms, len(remaining_vars)))
                    new_vars = random.sample(list(remaining_vars), num_new)
                    remaining_slots = max(0, max_terms - num_new)
                    additional_vars = random.sample(list(used_vars), min(remaining_slots, len(used_vars)))
                    eq_vars = new_vars + additional_vars
                    random.shuffle(eq_vars)
                else:
                    eq_vars = random.sample(list(vars), random.randint(1, min(num_vars, max_terms)))
                
                lhs = cls._generate_expression(eq_vars[:len(eq_vars)//2], operations, coef_range)
                rhs = cls._generate_expression(eq_vars[len(eq_vars)//2:], operations, coef_range)
                equations.append(Equation(f"{lhs} = {rhs}"))
                used_vars.update(eq_vars)
            
            structure = cls(equations)
            if structure.is_structure(count):
                if not force_all_vars or len(structure.vars) == num_vars:
                    return structure
        raise RuntimeError(f"Не удалось сгенерировать структуру за {max_attempts} попыток.")
    
    @classmethod
    def _generate_expression(cls, vars, operations, coef_range):
        '''
        Генерируем случайное выражение
        '''
        if not vars:
            return str(random.randint(*coef_range))
        
        expr = str(vars[0])
        if random.random() > 0.5:
            expr = f"{random.randint(*coef_range)}*{expr}"
        
        for var in vars[1:]:
            op = random.choice(operations)
            term = str(var)
            if random.random() > 0.5:
                term = f"{random.randint(*coef_range)}*{term}"
            
            if op in '*/' and random.random() > 0.7:
                expr = f"({expr})"
            
            expr = f"{expr} {op} {term}"
        
        expr = expr.replace("1*", "")
        return expr

    
    def exogenous(self):
        exogenous = set()
        for eq in self.equations:
            if len(eq.get_vars()) == 1:
                exogenous = exogenous.union(eq.get_vars())
        return exogenous

    def endogenous(self):
        return self.vars.difference(self.exogenous())
    
    def is_structure(self, count=False):
        for subset in powerset(self.equations):
            if subset:
                eq_num = len(subset)
                vars_in_subset = set().union(*(map(lambda x: set(x.vars), subset)))
                var_num = len(vars_in_subset)
                if eq_num > var_num:
                    return False
        if not(count):
            for subset in powerset(self.equations):
                if not subset:
                    continue
                    
                vars_in_subset = set().union(*(map(lambda x: set(x.vars), subset)))
                # считаем количество уравнений и переменных в subset
                eq_num = len(subset)
                var_num = len(vars_in_subset)
                
                if var_num > eq_num:
                    # количество переменных, которые будем фиксировать
                    number_fixed_var = var_num - eq_num
                    
                    # перебираем все варианты переменных для фиксации 
                    for fixed_vars in combinations(vars_in_subset, number_fixed_var):
                        remaining_vars = vars_in_subset - set(fixed_vars)
                        
                        # создаем временную систему уравнений 
                        temp_eqs = []
                        for eq in subset:
                            # убираем уравнения, если в них все переменные фиксированы
                            if all(v in fixed_vars for v in eq.vars):
                                continue
                            
                            # создаем уравнение с оставшимися переменными
                            remaining_vars_in_eq = set(eq.vars) - set(fixed_vars)
                            if remaining_vars_in_eq:
                                temp_eq = Equation("0")
                                temp_eq.vars = list(remaining_vars_in_eq)
                                temp_eqs.append(temp_eq)
                        
                        # создаем временную структуру
                        temp_structure = Structure(temp_eqs)
                        
                        # проверяем, что все переменные во временной структуре стали эндогенными
                        endogenous_vars = temp_structure.endogenous()
                        if not remaining_vars.issubset(endogenous_vars):
                            return False   
        return True
            
        
    def is_complete(self):
        return len(self.equations) == len(self.vars)

    def is_minimal_structure(self):
        """
        Проверяет, является ли текущая структура минимальной
        """
        # структура не полная -> не минимальная
        if not self.is_complete():
            return False
        
        # в структуре только одно уравнение - она минимальна
        if len(self.equations) == 1:
            return True
        
        for subset in powerset(self.equations):
            if not subset or len(subset) == len(self.equations):
                continue
                
            temp_structure = Structure(equations=subset)
            left_vars = set().union(*[eq.equation.free_symbols for eq in subset])
            temp_structure.vars = left_vars
            
            # если найдено полное подмножество - текущая структура не минимальна
            if temp_structure.is_complete():
                return False
        
        # если не найдено полных подмножеств - структура минимальна
        return True

    def build_causal_mapping_with_counter(self, counter=False):
        if not self.is_complete():
            raise ValueError("Структура не является полной")
        
        op_count = 0
        var_counts = {}
        for eq in self.equations:
            for var in eq.vars:
                var_counts[var] = var_counts.get(var, 0) + 1
                op_count += 1

        # строим граф зависимостей между переменными
        causal_graph = {var: set() for var in self.vars}
        op_count += len(self.vars)
        
        for eq in self.equations:
            # для каждого уравнения находим переменную с минимальным числом вхождений
            vars_in_eq = sorted(eq.vars, key=lambda v: var_counts[v])
            effect_var = vars_in_eq[0]
            cause_vars = set(vars_in_eq[1:])
            
            causal_graph[effect_var].update(cause_vars)
            op_count += len(vars_in_eq) * 2

        closure = {var: set(deps) for var, deps in causal_graph.items()}
        op_count += len(closure)
        
        changed = True
        while changed:
            changed = False
            for var in closure:
                new_deps = set()
                for cause in closure[var]:
                    new_deps.update(closure[cause])
                    op_count += len(closure[cause])
                
                if not new_deps.issubset(closure[var]):
                    closure[var].update(new_deps)
                    changed = True
                    op_count += len(new_deps)
        if counter:
            return causal_graph, op_count
        else:
            return causal_graph
        

    def transitive_closure(self, causal_map=None):
        if causal_map is None:
            causal_map = self.build_causal_mapping()
        
        closure = {var: set(deps) for var, deps in causal_map.items()}
        changed = True
        
        while changed:
            changed = False
            for var in closure:
                new_deps = set()
                for cause in closure[var]:
                    new_deps.update(closure[cause])
                
                if not new_deps.issubset(closure[var]):
                    closure[var].update(new_deps)
                    changed = True
        
        return closure
    
    def get_all_causal_dependencies(self):
        try:
            causal_map = self.build_causal_mapping_with_counter()
            return self.transitive_closure(causal_map)
        except ValueError as e:
            print(f"Ошибка: {e}")
            return None

    def count_all_occurrences(self):
        var_counts = defaultdict(int)
        total = 0
    
        for eq in self.equations:
            lhs, rhs = eq.formula.split('=', 1)
            for part in [lhs, rhs]:
                try:
                    expr = sympify(part)
                    for atom in expr.atoms():
                        if isinstance(atom, Symbol):
                            count = expr.count(atom)
                            var_counts[atom.name] += count
                            total += count
                except:
                    continue
        return total

In [ ]:
eq1 = Equation("x = y + z")
eq2 = Equation("y = z + w")
eq3 = Equation("z = 2")
eq4 = Equation("w = 1")
struct = Structure([eq1, eq2, eq3, eq4])

full_deps = struct.get_all_causal_dependencies()

for var, causes in full_deps.items():
    print(f"{var} зависит от: {causes}")

In [ ]:
random_struct = Structure.generate_random(num_vars=10, num_equations=3)

print("Variables:", [str(v) for v in random_struct.vars])
print("Equations:")
for eq in random_struct.equations:
    print(eq.formula)

random_struct.count_all_occurrences()

In [ ]:
if __name__ == '__main__':

    tex1 = r"f_1(x_1)=0"
    tex2 = r"f_2(x_2)=0"
    tex3 = r"f_3(x_3)=0"
    tex4 = r"x_1+x_2+x_3+x_4+x_5=0"
    tex5 = r"x_1 + 6*x_3+x_4+x_5=0"
    tex6 = r"f_6(x_4, x_6)=0"
    tex7 = r"f_7(x_5, x_7)=0"

    e1 = Equation(formula=tex1)
    e2 = Equation(formula=tex2)
    e3 = Equation(formula=tex3)
    e4 = Equation(formula=tex4)
    e5 = Equation(formula=tex5)
    e6 = Equation(formula=tex6)
    e7 = Equation(formula=tex7)

    equations = [e1, e2, e3, e4, e5, e6, e7]
    s = Structure(equations)

In [ ]:
s.is_structure()

In [ ]:
s.is_minimal_structure()

In [ ]:
tex1 = r"f_1(x_1)=0"
tex2 = r"f_2(x_2)=0"
tex3 = r"f_3(x_3)=0"
tex4 = r"x_1+x_2+x_3+x_4+x_5=0"
tex5 = r"x_1 + 6*x_3+x_4+x_5=0"
tex6 = r"f_6(x_4, x_6)=0"
tex7 = r"f_7(x_5, x_7)=0"

e1 = Equation(formula=tex1)
e2 = Equation(formula=tex2)
e3 = Equation(formula=tex3)
e4 = Equation(formula=tex4)
e5 = Equation(formula=tex5)
e6 = Equation(formula=tex6)
e7 = Equation(formula=tex7)

equations = [e1, e2, e3, e4, e5, e6, e7]
s = Structure(equations)
print(s.build_causal_mapping_with_counter(counter=True))

In [ ]:
def analyze_operations(max_n=10):
    sizes = []
    actual_ops = []
    theoretical_ops = []
    theoretical_ops2 = []
    theoretical_ops3 = []

    for n in range(1, max_n + 1, 10):
        struct = Structure.generate_random(num_vars=n, num_equations=n, count=True, max_terms=max_n)
        _, operations = struct.build_causal_mapping_with_counter(counter=True)
        
        sizes.append(n)
        actual_ops.append(operations)
        theoretical_ops.append(len(struct.vars)**(0.5)*struct.count_all_occurrences())
        theoretical_ops2.append(len(struct.vars)**(0.5)*(struct.count_all_occurrences())**2)
        theoretical_ops3.append(len(struct.vars)*(struct.count_all_occurrences()))
    
    plt.figure(figsize=(10, 5))
    plt.plot(sizes, actual_ops, 'bo-', label='Фактические операции')
    plt.plot(sizes, theoretical_ops, 'r--', label='Теоретическое количество операций')
    #plt.plot(sizes, theoretical_ops2, 'r', label='S^2 * sqrt(V)')
    plt.plot(sizes, theoretical_ops3, 'r', label='S * V')
    plt.xlabel('Размер структуры n')
    plt.ylabel('Количество операций')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
analyze_operations(100)